# Function calling - External Tool usage
![alt text](images/image2.png)

In [77]:
from dotenv import load_dotenv
load_dotenv()
import os


''

In [38]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [39]:
from langchain_core.tools import tool
import datetime

@tool
def fake_weather_api(location:str) -> str:
    """ 
    Check the wheather in a given location on a given date.

    Args: 
    location: The location for which to retrieve weather data.
    
    Returns:
    The weather data for the specified location and date.
    """
    # return random weather data in the below format "Sunny, 22' C" 

    import random
    weather_conditions = ["Sunny", "Cloudy", "Rainy", "Snowy", "Windy"]
    # based on weather conditions generate a random degree in celcis
    temperature = random.randint(10, 30)
    return f"{random.choice(weather_conditions)},{temperature} C"
# create a list of tools
tools = [fake_weather_api]


In [20]:
fake_weather_api("France")

'Snowy,29 C'

In [22]:
llm_with_tools = llm.bind_tools(tools)
response = llm_with_tools.invoke("What is the weather in France today?")

In [23]:
response


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vhdr4EoPqUt90EGu3dFoQRND', 'function': {'arguments': '{"location":"France"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 91, 'total_tokens': 106, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-561e61f7-b7f6-4aab-8f85-3a114e99ec48-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'location': 'France'}, 'id': 'call_vhdr4EoPqUt90EGu3dFoQRND', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 15, 'total_tokens': 106, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [26]:
response.tool_calls

[{'name': 'fake_weather_api',
  'args': {'location': 'France'},
  'id': 'call_vhdr4EoPqUt90EGu3dFoQRND',
  'type': 'tool_call'}]

In [27]:
from langchain_core.messages import HumanMessage, ToolMessage
messages = [
    HumanMessage("How will the weather be in France today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)
messages

[HumanMessage(content='How will the weather be in France today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XnqDmjK59YzKw5CHjcZnPtic', 'function': {'arguments': '{"location":"France"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 100, 'total_tokens': 115, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b19a9159-f2ac-4e77-99e3-47e6330b1bad-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'location': 'France'}, 'id': 'call_XnqDmjK59YzKw5CHjcZnPtic', 'type': 'tool_call'}], usage_metadata={'input_tokens': 100, 'output_tokens': 15, 'total_tokens': 115, 'input

In [28]:
tool_mapping ={
    "fake_weather_api":fake_weather_api
}
tool_mapping

{'fake_weather_api': StructuredTool(name='fake_weather_api', description='Check the wheather in a given location on a given date.\n\nArgs: \nlocation: The location for which to retrieve weather data.\n\nReturns:\nThe weather data for the specified location and date.', args_schema=<class 'langchain_core.utils.pydantic.fake_weather_api'>, func=<function fake_weather_api at 0x000001827F2999E0>)}

In [29]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output =tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output,tool_call_id=tool_call["id"]))

In [30]:
tool_output

'Rainy,25 C'

In [31]:
messages

[HumanMessage(content='How will the weather be in France today? I would like to eat outside if possible', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XnqDmjK59YzKw5CHjcZnPtic', 'function': {'arguments': '{"location":"France"}', 'name': 'fake_weather_api'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 100, 'total_tokens': 115, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b19a9159-f2ac-4e77-99e3-47e6330b1bad-0', tool_calls=[{'name': 'fake_weather_api', 'args': {'location': 'France'}, 'id': 'call_XnqDmjK59YzKw5CHjcZnPtic', 'type': 'tool_call'}], usage_metadata={'input_tokens': 100, 'output_tokens': 15, 'total_tokens': 115, 'input

In [32]:
llm_with_tools.invoke(messages)

AIMessage(content='The weather in France today is rainy with a temperature of 25°C. It might not be the best day to eat outside.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 129, 'total_tokens': 156, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cc3043c5-c9ed-4bf8-b694-caf31dc114f2-0', usage_metadata={'input_tokens': 129, 'output_tokens': 27, 'total_tokens': 156, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

### Langchain also provides pre-defined tools 

In [81]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_community.tools.tavily_search import TavilySearchResults

#tool_output = TavilySearchResults(tavily_api_key=tavily_api_key, max_tokens=300)